In [1]:
!pip install seaborn

## State of the art (ROAD, sort base ranking and linear noisy imputation notrain method)

In [30]:

notretrain_threshold_linear = "/workspaces/outputs/road_8_exp_mothods/sort/results/noretrain.json"

In [31]:
import json
import pandas as pd
import numpy as np

In [32]:
with open(notretrain_threshold_linear) as json_file:
    notretrain_threshold_linear = json.load(json_file)

In [33]:
def averaging_accuracy(dict_result):
    import copy
    dictionary = copy.deepcopy(dict_result)
    for method in dictionary['imputations']:
        for key_m, value_m in dictionary[method].items():
            for key_sb, value_ac in value_m.items():
                for key_f, value_f in  value_ac.items():                  
                    for key_p, value_ar in value_f.items():
                        value_f[key_p] = sum(value_ar)/len(value_ar)
    return dictionary

In [34]:
import numpy
def rankdata(numbers , morf=False):
    array = numpy.array(numbers)
    if not morf:
        array = 1 - array
    temp = array.argsort()
    ranks = numpy.empty_like(temp)
  
    ranks[temp] = numpy.arange(len(array)) + 1
    return ranks

In [35]:
def ranker(data_dict, appraoch="zero", morf=True, items=["ig" , "gb"]):
    import copy
    if morf:
        method = 'MoRF'
    else:
        method ='LeRF'
    printable_data_dict = copy.deepcopy(data_dict)
    new_dict = {}
    ranked_dict = {}
    new_dict['x_axis'] = printable_data_dict["percentages"][0:7] #0:7
    for key_m,  val_m in printable_data_dict[appraoch].items():
        if key_m in items:
            for key_var, val_var in val_m.items():
                if not morf:
                    accs = val_var['lerf']
                    l = list(accs.values())
                    new_dict[f"{key_m}-{key_var}"] = l[0:7]
                else:
                    accs = val_var['morf']
                    new_dict[f"{key_m}-{key_var}"] = list(accs.values())[0:7]
    x = new_dict["x_axis"]
    del(new_dict["x_axis"])
        
    count = 0
    for each in x:
        list_item = []
        for k, v in new_dict.items():
            list_item.append(v[count])
        ranked_dict[each] = rankdata(list_item, morf=morf)
        count+=1
    return ranked_dict

In [48]:
def plot_consistancy( data_dict, appraoch="zero", morf=True, path=None, items=["ig"]):
    import copy
    if morf:
        method = 'MoRF'
    else:
        method ='LeRF'
    printable_data_dict = copy.deepcopy(data_dict)
    new_dict = {}
    new_dict['x_axis'] = printable_data_dict["percentages"][0:-1]
    for key_m,  val_m in printable_data_dict[appraoch].items():
        if key_m in items:
            for key_var, val_var in val_m.items():
                if not morf:
                    accs = val_var['lerf']
                    l = list(accs.values())
                    #l.reverse()
                    print(key_m)
                    new_dict[f"{key_m}-{key_var}"] = l[0:-1]

                else:
                    accs = val_var['morf']
                    new_dict[f"{key_m}-{key_var}"] = list(accs.values())[0:-1]
                 
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    #sns.set_theme(style="whitegrid")
    sns.set(font_scale=2.1, style="whitegrid")
    data = pd.DataFrame.from_dict(new_dict)
    data.rename(columns={'gb-base': 'GB', 'gb-sg': 'GB-SG', 'gb-var': 'GB-VAR', 'gb-sq': 'GB-SQ', 'ig-base': 'IG', 'ig-sg': 'IG-SG', 'ig-var': 'IG-VAR', 'ig-sq': 'IG-SQ'}, inplace=True)
    data = data.set_index('x_axis')

    ax = sns.lineplot(data=data, palette="tab10", linewidth=2.5)
    #ax.legend(loc='lower left',ncol=2)
    ax.set(ylim=(0.0, 1.0))
    ax.set(xlim=(0.0, 0.9))
    ax.set(xlabel=f'Ratio of pixels', ylabel='Accuracy')
    ax.legend(fontsize=13.8, loc='lower left',ncol=2)
    ax.figure.savefig(f'{path}/{appraoch}-{method}.pdf', bbox_inches='tight')
    ax.figure.clf()

In [37]:
from scipy.stats import spearmanr
def calculate_spearman_morf_lerf(morf, lerf):
    scores =[]
    for ind in morf.keys():
        score = spearmanr(morf[ind], lerf[ind]).correlation
        scores.append(score)
    return scores


In [38]:
def show_images(path):
    from PIL import Image
    import glob
    images = glob.glob(f'{path}/*.png')
    for im in images:
        im = Image.open(im)
        display(im)

In [39]:
averaged_accuracy_threshold_linear = averaging_accuracy(notretrain_threshold_linear)

## Liner fixed

## Sort (Linear)

In [50]:
plot_consistancy(averaged_accuracy_threshold_linear, morf=False, appraoch="linear", path=f'/workspaces/outputs/road_8_exp_mothods/ig/sort/plots/all')
#show_images(f'/workspaces/outputs/road_8_exp_mothods/sort/plots')

ig
ig
ig
ig


<Figure size 640x480 with 0 Axes>

0.5816326530612245


## SORT(Linear)

In [16]:
import numpy as np
ranking_lerf = ranker(averaged_accuracy_threshold_linear, morf=False, appraoch="linear")
ranking_morf = ranker(averaged_accuracy_threshold_linear, morf=True, appraoch="linear")
score = calculate_spearman_morf_lerf(ranking_lerf, ranking_morf)
print (np.mean(score))

0.5816326530612245


# SORT (Telea)

In [17]:
import numpy as np
ranking_lerf = ranker(averaged_accuracy_threshold_linear, morf=False, appraoch="telea")
ranking_morf = ranker(averaged_accuracy_threshold_linear, morf=True, appraoch="telea")
score = calculate_spearman_morf_lerf(ranking_lerf, ranking_morf)
print (np.mean(score))

0.6190476190476192


# SORT (NS)

In [18]:
import numpy as np
ranking_lerf = ranker(averaged_accuracy_threshold_linear, morf=False, appraoch="ns")
ranking_morf = ranker(averaged_accuracy_threshold_linear, morf=True, appraoch="ns")
score = calculate_spearman_morf_lerf(ranking_lerf, ranking_morf)
print (np.mean(score))

0.5646258503401361
